<a href="https://colab.research.google.com/github/sheon1206/Weekly_Reading_Review/blob/main/%EB%8F%85%ED%95%B4_%EC%A3%BC%EA%B0%84%ED%85%8C%EC%8A%A4%ED%8A%B8_%EC%83%9D%EC%84%B1%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# [1] 라이브러리 설치 및 폰트 설정
# 1. reportlab 설치 (이 줄이 꼭 필요합니다!)
!pip install reportlab

import os
import urllib.request
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.pdfbase.pdfmetrics import registerFontFamily

def setup_fonts():
    print("폰트 설정을 시작합니다...")

    # 나눔고딕 폰트 다운로드 URL
    font_urls = {
        'NanumGothic.ttf': 'https://github.com/google/fonts/raw/main/ofl/nanumgothic/NanumGothic-Regular.ttf',
        'NanumGothicBold.ttf': 'https://github.com/google/fonts/raw/main/ofl/nanumgothic/NanumGothic-Bold.ttf'
    }

    for font_name, url in font_urls.items():
        if not os.path.exists(font_name):
            print(f"다운로드 중: {font_name}...")
            try:
                urllib.request.urlretrieve(url, font_name)
            except Exception as e:
                print(f"다운로드 실패 ({font_name}): {e}")
                return False

    try:
        # 폰트 등록
        pdfmetrics.registerFont(TTFont('NanumGothic', 'NanumGothic.ttf'))
        pdfmetrics.registerFont(TTFont('NanumGothicBold', 'NanumGothicBold.ttf'))
        # Bold 처리를 위한 패밀리 등록
        registerFontFamily('NanumGothic', normal='NanumGothic', bold='NanumGothicBold')
        print("성공! 폰트 등록이 완료되었습니다.")
        return True
    except Exception as e:
        print(f"폰트 등록 중 에러 발생: {e}")
        return False

# 실행
setup_fonts()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.9 MB/s eta 0:00:00
폰트 설정을 시작합니다...
다운로드 중: NanumGothic.ttf...
다운로드 중: NanumGothicBold.ttf...
성공! 폰트 등록이 완료되었습니다.


True

In [7]:
from google.colab import drive
# 1. 구글 드라이브 마운트 (이 코드가 실행되면 팝업창에서 권한 허용을 해야 합니다)
drive.mount('/content/drive')
import json, os, re
from reportlab.lib.pagesizes import A4
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import mm
from reportlab.platypus import (
    BaseDocTemplate, Frame, PageTemplate, Paragraph, Spacer,
    Table, TableStyle, PageBreak, FrameBreak, NextPageTemplate,
    KeepTogether, HRFlowable
)
from reportlab.lib.enums import TA_JUSTIFY, TA_CENTER, TA_LEFT

# [설정]
BASE_DIR = '/content/drive/MyDrive/weekly_reading_test_data'
if not os.path.exists(BASE_DIR):
    os.makedirs(BASE_DIR)
    print(f"폴더가 없어서 새로 만들었습니다: {BASE_DIR}")

INPUT_FILE = os.path.join(BASE_DIR, 'test_data.json')
OUTPUT_FILE = os.path.join(BASE_DIR, 'Weekly_Reading_Review.pdf')

# [스타일 & 유틸]
def get_custom_styles():
    s = getSampleStyleSheet()
    s['Normal'].fontName, s['Normal'].fontSize, s['Normal'].leading = 'NanumGothic', 10, 14

    # [Fix] fontName 충돌 방지 로직 적용
    def add_style(name, parent, **kwargs):
        kwargs.setdefault('fontName', 'NanumGothic')
        s.add(ParagraphStyle(name=name, parent=s[parent], **kwargs))

    add_style('MainTitle', 'Heading1', fontName='NanumGothicBold', fontSize=20, alignment=TA_CENTER, spaceAfter=15)
    add_style('PassageTitle', 'Heading2', fontName='NanumGothicBold', fontSize=11, spaceAfter=5, textColor=colors.darkblue)
    add_style('PassageBox', 'Normal', fontSize=9.5, leading=16, alignment=TA_JUSTIFY, backColor=colors.whitesmoke, borderWidth=0.5, borderColor=colors.gray, borderPadding=8, spaceAfter=10)
    add_style('QuestionText', 'Normal', fontName='NanumGothicBold', fontSize=10, spaceAfter=3)
    add_style('ContextText', 'Normal', fontSize=9.5, alignment=TA_LEFT, backColor=colors.whitesmoke, borderWidth=0.5, borderColor=colors.lightgrey, borderPadding=6, leftIndent=5, rightIndent=5, spaceAfter=5)
    add_style('OptionText', 'Normal', fontSize=9, leftIndent=10, spaceAfter=1)
    add_style('VocabWord', 'Normal', fontName='NanumGothicBold', fontSize=9, alignment=TA_CENTER)
    add_style('AnswerTitle', 'Heading2', fontName='NanumGothicBold', fontSize=14, spaceBefore=15, spaceAfter=10, textColor=colors.darkblue)
    add_style('ExplanationBox', 'Normal', fontSize=9.5, leading=15, spaceAfter=10)
    return s

def clean_opt(text, idx):
    text = re.sub(r'^[①-⑩❶-❿➊-➓]\s*|^\(\d+\)\s*|^\d+\.\s*', '', str(text))
    return f"{idx+1}. {text}"

def fmt_ans(val, mode='text'):
    if mode == 'vocab_group': return "단어확인" if mode == 'table' else "(아래 표 참조)"
    val = str(val).strip() if val else ""
    if mode == 'table' and len(val) > 6: return "서술형"
    return val

def safe_sort_key(q):
    try:
        val = str(q.get('id', '0'))
        if '-' in val: return int(val.split('-')[1])
        return int(val)
    except: return 0

# [메인 로직]
def create_pdf():
    if not os.path.exists(INPUT_FILE): return print(f"파일 없음: {INPUT_FILE}")

    with open(INPUT_FILE, 'r', encoding='utf-8') as f:
        data = json.loads(re.sub(r'^```json\s*|\s*```$', '', f.read()))

    # [추가] 총 문항 수 미리 계산 (Score 표시용)
    total_qs = 0
    for p in data.get('passages', []):
        qs = p.get('questions', {})
        # Vocabulary (그룹이 있다면 그룹 개수를 1개로 칠지, 내부 아이템 수를 셀지에 따라 다름. 여기선 그룹 ID 기준 1개로 카운트)
        for v in qs.get('vocabulary', []):
            if v.get('type') == 'vocabulary_group':
                total_qs += 1
        total_qs += len(qs.get('reading', []))
        total_qs += len(qs.get('syntax_grammar', []))
        total_qs += len(qs.get('logic_flow', []))

    # 레이아웃 설정
    doc = BaseDocTemplate(OUTPUT_FILE, pagesize=A4, margin=15*mm)
    page_w, page_h = A4
    margin, gap = 15*mm, 8*mm
    col_w = (page_w - 2*margin - gap) / 2
    full_width = page_w - 2*margin  # 헤더용 전체 너비

    frames = {
        'title': Frame(margin, page_h-margin-35*mm, page_w-2*margin, 35*mm, id='title', showBoundary=0),
        'c1_first': Frame(margin, margin, col_w, page_h-2*margin-40*mm, id='c1_first', showBoundary=0),
        'c2_first': Frame(margin+col_w+gap, margin, col_w, page_h-2*margin-40*mm, id='c2_first', showBoundary=0),
        'c1_full': Frame(margin, margin, col_w, page_h-2*margin, id='c1_full', showBoundary=0),
        'c2_full': Frame(margin+col_w+gap, margin, col_w, page_h-2*margin, id='c2_full', showBoundary=0)
    }

    doc.addPageTemplates([
        PageTemplate(id='FirstPage', frames=[frames['title'], frames['c1_first'], frames['c2_first']]),
        PageTemplate(id='LaterPage', frames=[frames['c1_full'], frames['c2_full']])
    ])

    story, s = [], get_custom_styles()
    info = data.get('test_info', {})

    # --- 헤더 수정됨 ---
    # 1. 메인 타이틀
    story.append(Paragraph(info.get('title', 'Weekly Test'), s['MainTitle']))

    # 2. 정보 란 (Date, Name, Score)
    # 빈칸 생성
    blank_line = "_" * 18
    header_data = [[
        f"Date: {blank_line}",
        f"Name: {blank_line}",
        f"Score: {'_'*6} / {total_qs}"
    ]]

    # 테이블 생성 (3열)
    h_tbl = Table(header_data, colWidths=[full_width*0.35, full_width*0.35, full_width*0.30])
    h_tbl.setStyle(TableStyle([
        ('FONTNAME', (0,0), (-1,-1), 'NanumGothic'),
        ('FONTSIZE', (0,0), (-1,-1), 10),
        ('ALIGN', (0,0), (0,0), 'LEFT'),    # Date 좌측 정렬
        ('ALIGN', (1,0), (1,0), 'CENTER'),  # Name 중앙 정렬
        ('ALIGN', (2,0), (2,0), 'RIGHT'),   # Score 우측 정렬
        ('VALIGN', (0,0), (-1,-1), 'BOTTOM'),
        ('LINEBELOW', (0,0), (-1,-1), 0.5, colors.black), # 하단 밑줄
        ('BOTTOMPADDING', (0,0), (-1,-1), 8), # 밑줄과 텍스트 사이 간격
    ]))
    story.append(h_tbl)

    # [삭제됨] description 출력 부분 제거
    # if info.get('description'): ...

    story.extend([FrameBreak(), NextPageTemplate('LaterPage')])

    # --- 문제 영역 ---
    all_qs_flat = [] # 정답지 생성을 위해 모든 문제 저장

    for idx, p in enumerate(data.get('passages', [])):
        if idx > 0: story.extend([Spacer(1, 5*mm), HRFlowable(width="100%", thickness=0.5, color=colors.gray, dash=[2,2]), Spacer(1, 5*mm)])

        story.append(Paragraph(f"[Passage {p.get('passage_id', idx+1)}] {p.get('source_title','')}", s['PassageTitle']))
        story.extend([Paragraph(p.get('passage_content','').replace('\n','<br/>'), s['PassageBox']), Spacer(1, 3*mm)])

        qs = p.get('questions', {})

        # 1. 어휘 (Vocabulary) - 별도 처리 (Sort 제외)
        vocab_list = qs.get('vocabulary', [])
        for v in vocab_list:
            if v.get('type') == 'vocabulary_group':
                q_id = v.get('id', 'V')
                block = [Paragraph(f"<b>{q_id}. {v.get('question','Vocabulary Check')}</b>", s['QuestionText']), Spacer(1, 2*mm)]
                items = v.get('items', [])
                t_data = [[Paragraph(i['word'], s['VocabWord']), "_____________"] for i in items]
                if t_data:
                    tbl = Table(t_data, colWidths=[col_w*0.4, col_w*0.6], style=[
                        ('GRID', (0,0), (-1,-1), 0.25, colors.lightgrey), ('VALIGN', (0,0), (-1,-1), 'MIDDLE'), ('PADDING', (0,0), (-1,-1), 3)
                    ])
                    block.append(tbl)
                block.append(Spacer(1, 4*mm))
                story.append(KeepTogether(block))
                all_qs_flat.append({'id': q_id, 'type': 'vocabulary_group', 'items': items, 'ans': None, 'expl': None})

        # 2. 독해 및 구문 (Reading & Syntax) - 번호순 정렬
        rs_list = qs.get('reading', []) + qs.get('syntax_grammar', []) + qs.get('logic_flow', [])
        rs_list.sort(key=safe_sort_key)

        for q in rs_list:
            q_id, q_type = q.get('id','?'), q.get('type','')
            block = [Paragraph(f"{q_id}. {q.get('question','')}", s['QuestionText'])]

            if q.get('context_sentence'):
                block.extend([Spacer(1, 1.5*mm), Paragraph(q['context_sentence'], s['ContextText']), Spacer(1, 1.5*mm)])

            for i, opt in enumerate(q.get('options', [])):
                block.append(Paragraph(clean_opt(opt, i), s['OptionText']))

            if 'subjective' in q_type:
                block.extend([Spacer(1, 2*mm), Paragraph("Answer: " + "_"*50, s['OptionText'])])

            block.append(Spacer(1, 4*mm))
            story.append(KeepTogether(block))
            all_qs_flat.append({'id': q_id, 'type': q_type, 'ans': q.get('answer'), 'expl': q.get('explanation')})

    # --- 정답 및 해설 (2단 유지) ---
    story.extend([PageBreak(), Paragraph("정답 및 해설", s['MainTitle']), Spacer(1, 5*mm)])

    # 1. 빠른 정답
    story.append(Paragraph("1. 빠른 정답", s['AnswerTitle']))
    chunk, q_tbl_data = 2, [['문항', '정답'] * 2]
    for i in range(0, len(all_qs_flat), chunk):
        row = []
        for q in all_qs_flat[i:i+chunk]:
            mode = 'table' if q['type'] != 'vocabulary_group' else 'vocab_group'
            row.extend([q['id'], fmt_ans(q.get('ans'), mode)])
        row.extend(['', ''] * (chunk - len(all_qs_flat[i:i+chunk])))
        q_tbl_data.append(row)

    q_tbl = Table(q_tbl_data, colWidths=[(col_w/4)*0.8, (col_w/4)*1.2] * 2) # col_w에 맞춤
    q_tbl.setStyle(TableStyle([
        ('FONTNAME', (0,0), (-1,-1), 'NanumGothic'), ('FONTSIZE', (0,0), (-1,-1), 9),
        ('ALIGN', (0,0), (-1,-1), 'CENTER'), ('VALIGN', (0,0), (-1,-1), 'MIDDLE'),
        ('GRID', (0,0), (-1,-1), 0.5, colors.grey), ('BACKGROUND', (0,0), (-1,0), colors.lightgrey)
    ]))
    story.extend([q_tbl, Spacer(1, 10*mm)])

    # 2. 상세 해설
    story.append(Paragraph("2. 문항별 상세 해설", s['AnswerTitle']))
    for q in all_qs_flat:
        header = f"<b>[{q['id']}번] 정답:</b> "
        if q['type'] == 'vocabulary_group':
            story.extend([Paragraph(header + "(아래 참조)", s['Normal']), Spacer(1, 2*mm)])
            v_data = [['단어', '뜻']] + [[i['word'], i['answer']] for i in q['items'] if 'answer' in i]
            # items에 answer가 없는 경우 대비 (혹시 모를 오류 방지)
            if len(v_data) == 1 and q['items']: # answer 키가 없는 경우 그냥 빈칸
                 v_data = [['단어', '뜻']] + [[i['word'], ''] for i in q['items']]

            v_tbl = Table(v_data, colWidths=[col_w*0.35, col_w*0.65], hAlign='LEFT', style=[
                ('FONTNAME', (0,0), (-1,-1), 'NanumGothic'), ('GRID', (0,0), (-1,-1), 0.5, colors.lightgrey),
                ('BACKGROUND', (0,0), (-1,0), colors.whitesmoke)
            ])
            story.extend([v_tbl, Spacer(1, 5*mm)])
        else:
            ans_str = fmt_ans(q.get('ans'))
            story.append(Paragraph(f"{header}<font color='blue'>{ans_str}</font>", s['Normal']))
            if q.get('expl'):
                story.extend([Spacer(1, 2*mm), Paragraph(f"<b>[해설]</b> {q['expl']}", s['ExplanationBox'])])
            story.extend([Spacer(1, 3*mm), HRFlowable(width="100%", thickness=0.2, color=colors.lightgrey, dash=[1,1]), Spacer(1, 3*mm)])

    doc.build(story)
    print(f"PDF 생성 완료: {OUTPUT_FILE}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
